![ERA-LOGO](data/logo.png)

## Initialise Python Environment
Run the cell bellow to get everything setup and ready to go! \
This can take some time, so sit back and wait a few minutes...

In [ ]:
%pip install nbformat ipywidgets numpy matplotlib pandas plotly astropy
from ERA import *

## Upload a file:
Run the cell below, then click "Upload" and select your data file.

In [ ]:
uploader = FileUploader()

## Processing the data:
Before we can analyse the data, we first need to check what was recorded.
Run the following code chunk to retrieve this information from the UVfits file.

In [ ]:
uploader.write_file()
data = ERAData("telescope-data.uvfits")

## Prepare the data
The data is currently in a format that the telescope understands, but we need to turn it into numbers that humans can understand.

In [ ]:
# -- Extract the visibilities as complex numbers --
vis = np.zeros(Nblts,dtype=np.complex64)
for i in range(Nblts):
    real = np.average( hdu[0].data[i][PCOUNT].take(indices=0, axis=NAXIS-1-1).take(indices=0, axis=NAXIS-1-2) ) # Take Real Part & first index of polarisation, average over all other axes
    imag = np.average( hdu[0].data[i][PCOUNT].take(indices=1, axis=NAXIS-1-1).take(indices=0, axis=NAXIS-1-2) ) # Take Imag Part & first index of polarisation, average over all other axes
    vis[i] = real + 1j*imag

# -- Remove Autos --
cross = np.argwhere( UVdist > 0 )[:,0]
UVdist = UVdist[cross]
UVWs = UVWs[cross,:]
vis = vis[cross]
Nblts = len(cross)
Nbls = int(len(cross)/Ntimes)
#Nbls = Nbls - Nant
#Nblts = Nbls*Ntimes

# -- Generate Antenna Names --
ant_name = hdu[1].data.field(0)         # Antenna Names
ants_in_baseline = []                   # Antenna's that make up each baseline/visibility (One could derive this from BASELINES, but the convention differs between software)
k = 0
for p in range(Nant):
    for q in range(p+1,Nant,1):
        ants_in_baseline.append( [ant_name[p],ant_name[q]] )
        k += 1

# Compute image resolution & npix
uvmax = np.max(UVdist) *(c/freq_center) # Max UV dist in metres
lam = c / freq_center
resolution = 1.2 * lam / uvmax
pixres = resolution / 3
npix = int(2/pixres)

# Define image coordinate system
l_axis = np.linspace(-1,1,num=npix)
m_axis = np.linspace(-1,1,num=npix)
l = np.tile( l_axis , (len(l_axis),1) )
m = np.tile( m_axis , (len(m_axis),1) ).T
r2 = l**2 + m**2; r2[ r2 >= 1 ] = 1
n = np.sqrt( 1 - r2 )

# Define RA,DEC coordinate system
def gen_radec(npix,obsra,obsdec):
    # Convert to radians
    obsra  = obsra*np.pi/180.0
    obsdec = obsdec*np.pi/180.0

    # Define direction cosine coords - (l,m,n) grid
    grid = np.moveaxis( np.array([n,l,m]) , 0, -1)

    # -- Rotate for Dec --
    roty = np.array([
      [ np.cos(obsdec) ,0  ,np.sin(obsdec)],
      [0                ,1  ,0              ],
      [-np.sin(obsdec) ,0  ,np.cos(obsdec)]
    ])
    # -- Rotate for RA --
    rotz = np.array([
      [np.cos(-obsra) ,-np.sin(-obsra) ,0 ],
      [np.sin(-obsra) , np.cos(-obsra) ,0 ],
      [0            ,0             ,1 ]
    ])

    rot  = np.matmul( roty , rotz )
    grid = np.matmul( grid , rot )

    ra  = np.arctan2(grid[:,:,1],grid[:,:,0]) * 180/np.pi
    dec = np.arctan2(grid[:,:,2], np.sqrt( grid[:,:,0]**2 + grid[:,:,1]**2 ) ) * 180/np.pi

    ra[ r2 >= 1 ] = 0
    dec[ r2 >= 1 ] = 0

    return ra, dec

ra_coords, dec_coords = gen_radec(npix,OBSRA,OBSDEC)



## Create some plotting functions
We need to turn the multidimensional data into something that we can understand in just two dimensions. To do this we create some special functions that let us project the data onto a pair of x and y axis.

In [ ]:
# ------ Imaging Functions ------
def grid_nearest(select=np.arange(Nblts,dtype=int)):
    Nvis = len(select) # The number of visibilities selected to include in the data

    VISgrid = np.zeros([npix,npix],dtype=np.complex64)
    WEIGHT  = np.zeros([npix,npix],dtype=int)
    for i in range(Nvis):
        x = int(np.round( UVWs[select[i],0]*2 ))
        y = int(np.round( UVWs[select[i],1]*2 ))
        VISgrid[y,x]   += vis[select[i]]
        VISgrid[-y,-x] += np.conj(vis[select[i]])
        WEIGHT[y,x]    += 1
        WEIGHT[-y,-x]  += 1
    WEIGHT[ WEIGHT == 0 ] = 1 # avoid dividing by 0
    VISgrid = VISgrid / WEIGHT  # Is this natural weighting?? IDK, maybe its uniform. Which one is best? I cant remember
    return VISgrid

def gen_image(select=np.arange(Nblts,dtype=int)):
    VISgrid = grid_nearest(select=select)
    img = np.fft.fftshift(np.fft.fftshift(np.fft.fft2( VISgrid ),axes=0),axes=1) / npix
    return np.abs( img )

# ------ Plotly Convenience Functions ------
def latitude_line(lat,dec=0,N=100):
    lat = lat/180.0*np.pi
    dec = dec/180.0*np.pi
    t = np.linspace(-np.pi,np.pi,N)
    line = np.array([
        np.cos(t)*np.cos(np.ones(N)*lat),
        np.sin(t)*np.cos(np.ones(N)*lat),
        np.sin(np.ones(N)*lat)
    ]).T
    rot = np.array([
        [ np.cos(-dec) ,0  ,np.sin(-dec)],
        [0            ,1  ,0          ],
        [-np.sin(-dec) ,0  ,np.cos(-dec)]
    ])
    line = np.matmul( line , rot )

    # --- remove points that are not visible ---
    idx = np.argwhere( line[:,0] > 0 )[:,0]
    line = line[idx,:]

    # --- Generate SVG path ---
    if len(line) == 0:
        return ""
    path = f'M {line[0,1]}, {line[0,2]}'
    for k in range(1,len(line)):
        path += f'L{line[k,1]}, {line[k,2]}'
    return path

def longitude_line(lon,dec=0,N=100):
    lon = lon/180.0*np.pi
    dec = dec/180.0*np.pi
    t = np.linspace(-np.pi,np.pi,N)
    line = np.array([
        np.cos(t),
        np.zeros(N),
        np.sin(t)
    ]).T
    rotz = np.array([
        [np.cos(-lon) ,-np.sin(-lon) ,0 ],
        [np.sin(-lon) , np.cos(-lon) ,0 ],
        [0            ,0             ,1 ]
    ])
    line = np.matmul( line , rotz )
    roty = np.array([
        [ np.cos(-dec) ,0  ,np.sin(-dec)],
        [0             ,1  ,0           ],
        [-np.sin(-dec) ,0  ,np.cos(-dec)]
    ])
    line = np.matmul( line , roty )

    # --- remove points that are not visible ---
    idx = np.argwhere( line[:,0] > 0 )[:,0]
    line = line[idx,:]

    # --- Generate SVG path ---
    if len(line) == 0:
        return ""
    path = f'M {line[0,1]}, {line[0,2]}'
    for k in range(1,len(line)):
        path += f'L{line[k,1]}, {line[k,2]}'
    return path

# Manualy Draw WCS Grid
def drawWCSGrid(fig):
    fig.add_shape(type="circle",
      xref="x", yref="y",
      x0=-1, y0=-1,
      x1=1, y1=1,
      opacity=1,
      fillcolor="rgba(0,0,0,0)",
      line_color="white",
      line_width=1
    )

    lat_resolution = 30
    for lat in np.arange(-90,90,lat_resolution):
        fig.add_shape(
            type="path",
            path=latitude_line(lat=lat,dec=OBSDEC),
            fillcolor="rgba(0,0,0,0)",
            line_color="white",
            line_width=1
        )

    lon_resolution = 30
    lon_offset = OBSRA % lon_resolution
    for lon in np.arange(-90,90,lon_resolution):
        fig.add_shape(
            type="path",
            path=longitude_line(lon=lon+lon_offset,dec=OBSDEC),
            fillcolor="rgba(0,0,0,0)",
            line_color="white",
            line_width=1
        )

#  Define a sqrt colorscale
viridis = px.colors.sequential.Viridis
colorscale = [
    [0.0**2, viridis[0]],
    [0.2**2, viridis[2]],
    [0.4**2, viridis[4]],
    [0.6**2, viridis[6]],
    [0.8**2, viridis[8]],
    [1.0**2, viridis[9]],
]


## Figure: UV Coverage

In [ ]:
# plot u,v coverage including all timesteps
fig = go.Figure(
    data = [
        go.Scattergl(
          name = "",
          x = UVWs[:,0]*(c/freq_center),
          y = UVWs[:,1]*(c/freq_center),
          text = [ ants_in_baseline[k][0] + " x " + ants_in_baseline[k][1] for k in range(Nbls) ]*Ntimes,
          mode = "markers",
          marker = dict(
              showscale = False, # disable colorbar
              size = 6
          )
        ),
        go.Scattergl(
          name = "conjugate",
          x = -UVWs[:,0]*(c/freq_center),
          y = -UVWs[:,1]*(c/freq_center),
          text = [ ants_in_baseline[k][1] + " x " + ants_in_baseline[k][0] for k in range(Nbls) ]*Ntimes,
          mode = "markers",
          marker = dict(
              showscale = False, # disable colorbar
              size = 6
          )
        )
    ],
    layout = {
        "template": "simple_white",
        "autosize": False,
        "width": 600,
        "height": 600,
        "xaxis": {
            "title_text": "East baseline [m]",
            "title_font": {"size": 20}
          },
        "yaxis": {
            "title_text": "North baseline [m]",
            "title_font": {"size": 20}
          },
        "showlegend": False
    }
)

fig

## Figure: Visibilities vs Baseline Length

In [ ]:
fig = go.Figure(
    data = go.Scattergl(
            name = "",
            x = UVdist*(c/freq_center),
            y = np.abs(vis),
            text = [ ants_in_baseline[k][0] + " x " + ants_in_baseline[k][1] for k in range(Nbls) ]*Ntimes,
            mode = "markers",
            marker = dict(
                showscale = False, # disable colorbar
                size = 6
            )
    ),
    layout = {
        "template": "simple_white",
        "autosize": False,
        "width": 800,
        "height": 600,
        "xaxis": {
            "title_text": "Baseline length [m]",
            "title_font": {"size": 20}
          },
        "yaxis": {
            "title_text": "Visibility amplitude [Jy]",
            "title_font": {"size": 20}
          },
        "showlegend": False
    }
)

fig

## Figure: All sky image

In [ ]:
# Generate Image
img = gen_image()

# Generate Figure
fig = go.Figure(
    data = [
      go.Heatmap(
        name = "",
        z = img,
        x = l_axis,
        y = m_axis,
        hovertemplate="RA: %{text[0]:,.2f} <br />DEC: %{text[1]:,.2f} <br />Flux Density: %{text[2]:,.3f} Jy",
        text = [[  [ ra_coords[y,x] , dec_coords[y,x] , img[y,x] ]  for x in range(npix) ] for y in range(npix) ],
        colorscale = colorscale
      )
    ],
    layout = {
        "template": "simple_white",
        "autosize": False,
        "width": 700,
        "height": 700,
        "xaxis": {
            "title_text": "",
            "title_font": {"size": 20}
          },
        "yaxis": {
            "title_text": "",
            "title_font": {"size": 20}
          }
    }
)

# --- Manualy Draw WCS Grid ---
drawWCSGrid(fig)

fig

## Figure: Image vs Baseline Length (Increasing Resolution)

In [ ]:
# Generate data for each animation frame
Nframes = 16
data = np.zeros([npix,npix,Nframes])
uvmax = np.ceil(np.max(UVdist) * (c/freq_center) ) / (c/freq_center)

for n in range(Nframes):
  uv_cutoff = (n+1)/Nframes * uvmax
  select = np.argwhere( UVdist <= uv_cutoff )[:,0]
  data[:,:,n] = gen_image(select=select)

# --- Define animation frames ---
frames = []
slider_steps = []
for n in range(Nframes):
  uv_cutoff = (n+1)/Nframes * uvmax *(c/freq_center) # (metres)

  # --- define data for this frame ---
  frames.append(
    go.Heatmap(
      name = "",
      z = data[:,:,n],
      x = l_axis,
      y = m_axis,
      colorscale = colorscale,
      visible = False #All frames not visible by default (We will manually enable frame 0 later)
    )
  )

  # --- Define plot title and slider related stuff for this frame ---
  slider_step = {
      "method": "restyle", #"update",
      "label": np.round(uv_cutoff,2),
      "args": [
                {"visible": [False] * Nframes}
              ]
  }
  slider_step["args"][0]["visible"][n] = True
  slider_steps.append(slider_step)

# --- Create Figure ---
fig = go.Figure(
    data=frames,
    layout = {
        "template": "simple_white",
        "autosize": False,
        "width": 700,
        "height": 750,
        "xaxis": {
            "title_text": "",
            "title_font": {"size": 20}
          },
        "yaxis": {
            "title_text": "",
            "title_font": {"size": 20}
          },
        "sliders": [{
            "active": 0,
            "currentvalue": {"prefix": "Max Baseline Length (m): "},
            "pad": {"t": 50},
            "steps": slider_steps
          }]
    }
)

# --- Manualy Draw WCS Grid ---
drawWCSGrid(fig)

fig.data[0].visible = True
fig.show()

## Figure: Image & Visibilities vs Basleine Length

In [ ]:
# Generate data for each animation frame
Nframes = 16
fig = make_subplots(rows=1, cols=2)

uvmax  = np.ceil(np.max(UVdist) * (c/freq_center) ) / (c/freq_center)
vismax = np.max(np.abs(vis)) * 1.05
vismin = np.min(np.abs(vis)) / 1.05

steps = []
for n in range(Nframes):
  uv_cutoff = (n+1)/Nframes * uvmax
  select = np.argwhere( UVdist <= uv_cutoff )[:,0]

  # --- Generate Data ---
  uv_data  = UVdist[select] * (c/freq_center)
  vis_data = np.abs(vis[select])
  img_data = gen_image(select=select)

  # --- Add UVplot Trace ---
  fig.add_trace(
      go.Scattergl(
          name = "",
          x = uv_data,
          y = vis_data,
          mode = "markers",
          marker = dict(
              color = "black",
              showscale = False, # disable colorbar
              size = 4
          ),
          visible = False #All frames not visible by default (We will manually enable frame 0 later)
      ),
      row=1, col=1
  )

  fig.add_trace(
      go.Heatmap(
        name = "",
        z = img_data,
        x = l_axis,
        y = m_axis,
        colorscale = colorscale,
        visible = False #All frames not visible by default (We will manually enable frame 0 later)
      ),
      row=1, col=2
  )

  # Define slide data
  step = {
      "method": 'restyle',
      "args": ['visible', ['legendonly'] * (2*Nframes) ],
      "label": np.round(uv_cutoff*(c/freq_center),2)
  }
  step['args'][1][2*n  ] = True
  step['args'][1][2*n+1] = True
  steps.append(step)

fig.update_layout(width=1100, height=600,autosize=False)
fig.layout["template"] = "simple_white"
fig.layout["sliders"] = [{
    "steps": steps,
    "currentvalue": {"prefix": "Max Baseline Length (m): "}
}]

for n in range(Nframes):
  fig.layout["xaxis"+str(n*2+1)] = {
      "range": [0,uvmax],
      "domain": [0,0.5]
  }
  fig.layout["yaxis"+str(n*2+1)] = {
      "range": [vismin,vismax],
      "domain": [0,1]
  }

fig.data[0].visible = True
fig.data[1].visible = True

fig.show()

## Figure: Image vs UV Coverage

In [ ]:
# Generate data for each animation frame
Nframes = 12
fig = make_subplots(rows=1, cols=2)

uvmax  = np.ceil(np.max(UVdist) * (c/freq_center) ) / (c/freq_center)

steps = []
for n in range(Nframes):
  uv_cutoff = (n+1)/Nframes * uvmax
  select = np.argwhere( UVdist <= uv_cutoff )[:,0]

  # --- Generate Data ---
  UVs = UVWs[select,0:2] * (c/freq_center)
  UVs = np.tile( UVs , (2,1) )
  UVs[int(UVs.shape[0]/2):,:] *= -1

  img_data = gen_image(select=select)

  # --- Add UVplot Trace ---
  fig.add_trace(
      go.Scattergl(
          name = "",
          x = UVs[:,0],
          y = UVs[:,1],
          mode = "markers",
          marker = dict(
              color = "black",
              showscale = False, # disable colorbar
              size = 4
          ),
          visible = False #All frames not visible by default (We will manually enable frame 0 later)
      ),
      row=1, col=1
  )

  fig.add_trace(
      go.Heatmap(
        name = "",
        z = img_data,
        x = l_axis,
        y = m_axis,
        colorscale = colorscale,
        visible = False #All frames not visible by default (We will manually enable frame 0 later)
      ),
      row=1, col=2
  )

  # Define slide data
  step = {
      "method": 'restyle',
      "args": ['visible', ['legendonly'] * (2*Nframes) ],
      "label": np.round(uv_cutoff*(c/freq_center),2)
  }
  step['args'][1][2*n  ] = True
  step['args'][1][2*n+1] = True
  steps.append(step)

fig.update_layout(width=1100, height=650,autosize=False)
fig.layout["template"] = "simple_white"
fig.layout["sliders"] = [{
    "steps": steps,
    "currentvalue": {"prefix": "Max Baseline Length (m): "}
}]

for n in range(Nframes):
  fig.layout["xaxis"+str(n*2+1)] = {
      "range": [-uvmax,uvmax],
      "domain": [0,0.5]
  }
  fig.layout["yaxis"+str(n*2+1)] = {
      "range": [-uvmax,uvmax],
      "domain": [0,1]
  }

fig.data[0].visible = True
fig.data[1].visible = True

fig.show()